In [1]:
#!/usr/bin/env python
import sys
import subprocess
import rospy
import csv
from PyQt5 import QtCore, QtGui, QtWidgets, uic
from std_msgs.msg import Float64, UInt16
from datetime import date, datetime

#above code imports the required libraries



#initalizes variables to store ros variables these variables are used to 
day = 'none'
time = 'none'
outerTemp = 0
outerHum = 0
innerTemp = 0
innerHum = 0
flowRate = 0
lightResistance = 0

#loads the ui file made in qtpy5
qtcreator_file  = "Sensor Display.ui" 
Ui_MainWindow, QtBaseClass = uic.loadUiType(qtcreator_file)

#inital csv file name
csvname = 'Run Off.csv'
#feildnames for headings of csv file
fieldnames = ['Date','Time','Outer Temprature (c)', 'Inner Temprature (c)', 'Outer Humidity', 'Inner Humidity', 'Light Reading', 'Flow L/min'] 
    

#the function below appends the data given by the ros subscriber to the 
def ot(data):
        global outerTemp
        outerTemp = "%.1f" %data.data
def oh(data):
        global outerHum
        outerHum = "%.1f" %data.data
def it(data):
        global innerTemp
        innerTemp = "%.1f" %data.data
def ih(data):
        global innerHum
        innerHum = "%.1f" %data.data
def lr(data):
        global lightResistance
        lightReading = data.data
        lightResistance = 1000-lightReading
def fr(data):
        global flowRate
        flowRate = "%.2f" %data.data
        
#creates a ros listener function
def listener():
        rospy.init_node('listener')
        rospy.Subscriber("outerTemp", Float64, ot)
        rospy.Subscriber("outerHum", Float64, oh)
        rospy.Subscriber("innerTemp", Float64, it)
        rospy.Subscriber("innerHum", Float64, ih)
        rospy.Subscriber("lightIntensity", UInt16, lr)
        rospy.Subscriber("flowrate",Float64, fr)

    
listener()    

class MyWindow(QtWidgets.QMainWindow, Ui_MainWindow):
    
    

    def __init__(self):
        QtWidgets.QMainWindow.__init__(self)
        Ui_MainWindow.__init__(self)
        self.setupUi(self)
        
        # creates a timer to update values. when the timer runs out the values
        # will be refreshed and stored in
        self.updateTimer = QtCore.QTimer()
        self.updateTimer.timeout.connect(self.sensorUpdate)
        self.updateTimer.timeout.connect(listener)
        self.updateTimer.start(1000)
        
        # when clicked the button will create a CSV file with the textbox above its
        # name as the title
        self.create.clicked.connect(self.createCSV)
        
        
        
        # this function sets each line edit with their corresponding values
        # it also writes the values to the CSV file the program has open
    def sensorUpdate(self):
        listener()
        self.outerTemp.setText(str(outerTemp))
        self.innerTemp.setText(str(innerTemp))
        self.outerHum.setText(str(outerHum))
        self.innerHum.setText(str(innerHum))
        self.lightReading.setText(str(lightResistance))
        self.flowReading.setText(str(flowRate))
        
    
        time = datetime.now()
        day = date.today()
        
        fTime = time.strftime("%H:%M")
        fDate = day.strftime("%d/%m/%Y")
        
        csvfile = open(csvname,'a',newline='')
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({'Date':fDate,'Time':fTime,'Outer Temprature (c)': outerTemp,'Inner Temprature (c)': innerTemp,
                'Outer Humidity': outerHum, 'Inner Humidity':innerHum, 'Light Reading':lightResistance, 'Flow mL/sec':flowRate})
        
        
    #creates a csv file from the text entered into the line edit above the button    
    def createCSV(self):
        global csvname 
        csvname = self.csvName.text()
        csvfile = open(csvname,'w',newline='')
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
# main method displays ui and sets exit method
if __name__ == "__main__":
    
    app = QtWidgets.QApplication(sys.argv)
    
    window = MyWindow()
    window.show()
    sys.exit(app.exec_())
    
    
    
   

SystemExit: 0

/home/callum/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
s